In [ ]:
%load_ext autoreload
%autoreload 2
import time, base64,ssl, os, re,json5, pprint,random , math, hashlib, inspect
import pandas as pd
from selenium import webdriver# Controls the browser
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By                   # Locators (ID, CLASS_NAME, XPATH, etc.)
from selenium.webdriver.support.ui import WebDriverWait       # Waits for elements to appear
from selenium.webdriver.support import expected_conditions as EC  # Conditions like "visible", "clickable"
from selenium.webdriver.chrome.options import Options
from urllib.parse import urljoin
from bs4 import BeautifulSoup
from app.utils import Helper
from app.operation_executor import OperationExecutor
import pandas as pd
import numpy as np

from io import StringIO

In [ ]:
path = "https://www.jkbank.com/interest-rates"
options = Options()
driver = webdriver.Chrome(options=options)

driver.get(path)
tabList = driver.find_elements(By.CSS_SELECTOR,".owl-nav-carousel li a")
print(f"Total tabs found: {len(tabList)}")
for tab in tabList:
    try:
        driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", tab)
        time.sleep(1) 

        ActionChains(driver).move_to_element(tab).perform()

        WebDriverWait(driver, 10).until(EC.element_to_be_clickable(tab))
        print(tab.get_attribute("innerText"))
        pprint.pprint(tab.get_attribute("outerHTML"))
        tab.click()
        time.sleep(1)
        
        pprint.pprint(tab.get_attribute("outerHTML"))
        print("==============================")
        
        # tables = driver.find_elements(By.TAG_NAME, "table")
        # for idx,table in enumerate(tables):
        #     content = table.get_attribute("outerHTML")
        #     with open(f"table_{idx}.html","w+",encoding="utf-8") as f:
        #         f.write(content)
    except Exception as e:
        print(f"Failed to click tab: {e}")
    
driver.quit()

In [ ]:
from app.operation_executor import OperationExecutor
from app.utils import Helper
from urllib.parse import urljoin
import os
ops = OperationExecutor()
pathList = [r"25-09-02T23-52-23_cache.json",r"25-09-03T16-21-30_cache.json"]

for path in pathList:
    
    root_file = os.path.basename(path).split("_")[0]
    function_to_execute = {
    "primary":[["normalize_df","value","norm_table","table_html"],["sha1","value","SHA_ONE","pdf"]],
    "secondary":[["sha1","norm_table","SHA_ONE"]],
    }
    try:
        data = Helper.load_json(path, typ="json5")
        processed_data = ops.runner(data,function_to_execute)
        Helper.save_json(processed_data,f"{root_file}_process.json")
    except Exception as e:
        print(f"\nError in Processing.. Skipping")
        continue

In [2]:
from app.operation_executor import OperationExecutor
from app.utils import Helper
ops = OperationExecutor()
yesterday = Helper.load_json("25-09-02T23-52-23_process.json")
today = Helper.load_json("25-09-03T16-21-30_process.json")

result = ops.process_comparison(yesterday,today,key="SHA_ONE")
Helper.save_json(result, "compare-030925.json")

ops.generate_sorted_excel_report(result)

'DepositRate_Comparison_Report.xlsx'

In [ ]:
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
import random, time, math, ssl

ssl._create_default_https_context = ssl._create_stdlib_context

def human_scroll(driver, total_scroll=2000, step=50):
    for y in range(0, total_scroll, step):
        driver.execute_script(f"window.scrollTo(0, {y});")
        time.sleep(random.uniform(0.05, 0.3))

def human_move_mouse(driver, start=(0, 0), end=(300, 300), steps=30):
    actions = ActionChains(driver)
    x1, y1 = start
    x2, y2 = end
    for t in range(steps):
        progress = t / steps
        x = x1 + (x2 - x1) * progress + random.uniform(-2, 2)
        y = y1 + (y2 - y1) * progress + math.sin(progress * math.pi) * 20
        actions.move_by_offset(int(x), int(y))
    actions.perform()

def human_click(elem):
    time.sleep(random.uniform(0.2, 1.5))
    elem.click()

profile_path = Helper.load_json("paths.json").get("profile_path")
profile_dir = Helper.load_json("paths.json").get("profile_name")

options = Options()
options.add_argument(f"--user-data-dir={profile_path}")
options.add_argument(f"--profile-directory={profile_dir}")
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("--start-maximized")

driver = uc.Chrome(options=options)

driver.get("https://www.unionbankofindia.co.in/en/details/rate-of-interest")

human_scroll(driver, total_scroll=1500, step=70)
human_move_mouse(driver, start=(0, 0), end=(200, 200), steps=40)

try:
    table = driver.find_element(By.TAG_NAME, "table")
    human_click(table) # Just demo click
    print("Table found:", table.get_attribute("outerHTML")[:200])
except Exception as e:
    print("No table found:", e)

time.sleep(5)
driver.quit()


In [ ]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import time

path = "https://www.nseindia.com/market-data/equity-derivatives-watch"
options = Options()
driver = webdriver.Chrome(options=options)
driver.get(path)

time.sleep(5)

select_element = driver.find_element(By.CSS_SELECTOR, "select.no-border-radius")
select = Select(select_element)

for option in select.options:
    try:
        value = option.get_attribute("value")
        print(f"Selecting: {option.text} ({value})")
        
        select.select_by_value(value)
        time.sleep(2)

        tables = driver.find_elements(By.CSS_SELECTOR, "#eqderivativesTable")
        print(f"Found {len(tables)} tables for {option.text}")
        
        for idx, table in enumerate(tables):
            content = table.get_attribute("outerHTML")
            with open(f"{value}_table_{idx}.html", "w+", encoding="utf-8") as f:
                f.write(content)

    except Exception as e:
        print(f"Failed to select {option.text}: {e}")from app.operation_executor import OperationExecutor

# path = "https://www.jkbank.com/interest-rates"
# options = Options()
# driver = webdriver.Chrome(options=options)

# driver.get(path)
# tabList = driver.find_elements(By.CSS_SELECTOR,".owl-nav-carousel li a")
# print(f"Total tabs found: {len(tabList)}")
# for tab in tabList:
#     try:
#         driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", tab)
#         time.sleep(1) 

#         ActionChains(driver).move_to_element(tab).perform()

#         WebDriverWait(driver, 10).until(EC.element_to_be_clickable(tab))
#         print(tab.get_attribute("innerText"))
#         tab.click()
#         time.sleep(1)
        
#         tables = driver.find_elements(By.TAG_NAME, "table")
#         # for idx,table in enumerate(tables):
#         #     content = table.get_attribute("outerHTML")
#         #     with open(f"table_{idx}.html","w+",encoding="utf-8") as f:
#         #         f.write(content)
#     except Exception as e:
#         print(f"Failed to click tab: {e}")
        
        
# path = "https://www.icicibank.com/personal-banking/deposits/fixed-deposit/fd-interest-rates?ITM=nli_cms_FD_rates_menu_navigation"
# options = Options()
# driver = webdriver.Chrome(options=options)

# driver.get(path)
# tabList = driver.find_elements(By.CSS_SELECTOR,".tabs-list button")
# print(f"Total tabs found: {len(tabList)}")
# for tab in tabList:
#     try:
#         driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", tab)
#         time.sleep(1) 

#         ActionChains(driver).move_to_element(tab).perform()

#         WebDriverWait(driver, 10).until(EC.element_to_be_clickable(tab))
#         print(tab.get_attribute("innerText"))
#         tab.click()
#         time.sleep(1)
        
#         tables = driver.find_elements(By.TAG_NAME, "table")
#         # for idx,table in enumerate(tables):
#         #     content = table.get_attribute("outerHTML")
#         #     with open(f"table_{idx}.html","w+",encoding="utf-8") as f:
#         #         f.write(content)
#     except Exception as e:
#         print(f"Failed to click tab: {e}")
